In [1]:
# Se importan Instaladores
#pip install matplotlib
#pip install seaborn

# Importar librerias

## Importar librerias de terceros

In [145]:
# Se importan librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import pickle


# Bajar y extraer archivos

La siguientes celdas solo se ejecutan si no se tiene el archivo cirrhosis.csv y no se puede obtener con dvc pull

## Definir fuente de los archivos, origen y destino

In [37]:
# Se declaran las variables url, destino_del_archivo_descargado y destino_de_archivo_extraido

url="https://archive.ics.uci.edu/static/public/878/cirrhosis+patient+survival+prediction+dataset-1.zip"
destino_del_archivo_descargado="../data/cirrhosis.zip"
destino_de_archivo_extraido="../data"


## Descargar El archivo

In [ ]:
urlretrieve(url,destino_del_archivo_descargado)

## Extraer el contenido del zip y borrar el zip

In [39]:
with ZipFile(destino_del_archivo_descargado) as zObject: 
                zObject.extractall(path=destino_de_archivo_extraido)

os.remove(destino_del_archivo_descargado)


# Leer los datos

## Pasar el archivo csv a un data frame de pandas

### Leer el csv y pasalo a data frame de pandas

In [ ]:
# Lectura de archivo
path_del_archivo_cirrhosis="../data/cirrhosis.csv"
df_cirrhosis = pd.read_csv(path_del_archivo_cirrhosis)
df_cirrhosis.head()

## 

# Limpieza de Datos

Las siguientes celdas se ejecutan si no se tiene el archivo de cirrhosis_clean.csv o se hacen cambios en la sección.

## Describir el data Frame para comprender la información

### Obtener el tamaño del data frame

In [ ]:
# Dimensiones del set de datos
print("Tamaño del DataFrame:", df_cirrhosis.shape)

### Ver de que tipo son las variables

In [ ]:
df_cirrhosis.dtypes

Vemos que tenemos 3 columnas de tipo entero, 10 de tipo flotante y 7 de tipo objeto

## Identificar valores nulos

### Obtener porcentajes de valores nulos

In [ ]:
# Identificar % de los datos con valores nulos
valores_Nullos_del_df_cirrhosis = df_cirrhosis.isna()
valores_Nullos_del_df_cirrhosis.mean()*100

Podemos observar que 12 de las columnas contienen valores nulos

## Eliminar valores Nulos

### Se eliminan valores nulos de la columna Drug

Se eliminan los pacientes que no quisieron participar en el estudio. Ninguno de dichos pacientes estaba tomando algún medicamento por ello en los datos vienen como nulos en la variable "Drug"

In [ ]:
# Eliminar la columna 
df_cirrhosis=df_cirrhosis.dropna(subset=['Drug'])
# Impirmir el df sin los nulos de Drug
df_cirrhosis.head()

### Guardar la version del set de datos limpio en csv, para poder mantener las versiones en dvc

#### Guardar el df sin nulos en columna drugs

In [58]:
cirrhosis_df_clean_path = '../data/cirrhosis_clean.csv'
df_cirrhosis.to_csv(cirrhosis_df_clean_path)

### Revisar el total de pacientes a revisar

In [ ]:
# Universo de pacientes a estudiar
valores_Nullos_del_df_cirrhosis_limpios = df_cirrhosis.isna()
valores_Nullos_del_df_cirrhosis_limpios.count()

### Observar el porcentaje de nulos restantes

In [ ]:
# Identifica nuevos porcentajes de valores nulos.
valores_Nullos_del_df_cirrhosis_limpios.mean()*100

Observamos que aun algunas columnas contiene valores nulos, sin embargo para poder decidir que hacer con ellos se tiene que hacer un analiísi más profundo, para saber si se van a quitar o se van a imputar.

# EDA

Estas celdas se ejecutan para realizar el analisis de entrenamiento, si no se hacen cambios no es necesario ejecturar estas celdas. Sin mebargo si no se tienen los archivos de prueba, validacion y entrenamiento en csv, se puede ejecutar, pero lo ideal es jalar los archivos con dvc.

### Separar los datos en entrenamiento alidación y pruebas

Se seperan los datos al inicio de EDA para evitar leer información ajena al entrenamiento y poder tomar decisiones basadas unicamente en el set de entrenamiento y no afectar al modelo con información externa.

In [69]:
df_cirrhosis = pd.read_csv('../data/cirrhosis_clean.csv').drop('Unnamed: 0',axis=1)
cirrhosis_Xtrain, cirrhosis_Xtest1, cirrhosis_ytrain, cirrhosis_ytest1 = train_test_split(df_cirrhosis.drop(columns=["Status"]),df_cirrhosis["Status"], test_size=0.3, random_state=42, stratify=df_cirrhosis["Status"])
cirrhosis_Xval, cirrhosis_Xtest, cirrhosis_yval, cirrhosis_ytest = train_test_split(cirrhosis_Xtest1,cirrhosis_ytest1, test_size=0.5, random_state=42, stratify=cirrhosis_ytest1)

### Revisamos el tamaño de los sets de entrenamiento, validación y pruebas

In [ ]:
print(f"El tamaño del set de entrenamiento es {cirrhosis_Xtrain.shape}")
print(f"El tamaño del set de prueba es {cirrhosis_Xtest.shape}")
print(f"El tamaño del set de validación es {cirrhosis_Xval.shape}")

Confirmamos que los datos se separaron de manera correcta

### Guardar set de entrenamiento en csv, para poder tener versiones en dvc

In [73]:
cirrhosis_X_train_path = '../data/cirrhosis_X_train.csv'
cirrhosis_Xtrain.to_csv(cirrhosis_X_train_path)
cirrhosis_y_train_path = '../data/cirrhosis_y_train.csv'
cirrhosis_ytrain.to_csv(cirrhosis_y_train_path)

### Guardar set de pruebas en csv, para poder tener versiones en dvc

In [74]:
cirrhosis_X_test_path = '../data/cirrhosis_X_test.csv'
cirrhosis_Xtest.to_csv(cirrhosis_X_test_path)
cirrhosis_y_test_path = '../data/cirrhosis_y_test.csv'
cirrhosis_ytest.to_csv(cirrhosis_y_test_path)

### Guardar se te validación en csv, para poder tener versiones en dvc

In [75]:
cirrhosis_X_val_path = '../data/cirrhosis_X_val.csv'
cirrhosis_Xval.to_csv(cirrhosis_X_val_path)
cirrhosis_y_val_path = '../data/cirrhosis_y_val.csv'
cirrhosis_yval.to_csv(cirrhosis_y_val_path)

### Describir las variables numericas del set de datos

In [ ]:
#Describir variables numericas
cirrhosis_Xtrain.describe().T

### Describir variables no numericas del set de datos

In [ ]:
cirrhosis_Xtrain.describe(include=["O"]).T

### Observaremos los datos de los valores nulos para poder comprender que hacer con ellos

In [ ]:
valores_Nulos_Cirrhosis_train = valoresaNullos(cirrhosis_Xtrain)
valores_Nulos_Cirrhosis_train.identificar_valores_nulos()
cirrhosis_Xtrain_null = valores_Nulos_Cirrhosis_train.get_isna()
cirrhosis_Xtrain[cirrhosis_Xtrain_null.any(axis=1)]

Aqui vemos que lo más probable los datos nulos en los diferentes componentes, puden deberse a un examen distinto realizado, o fallas en la lectura en el analisis médico hecho. Por eso tenemos dos opciones, eliminar las lecturas fallidas o remplazar las lecturas fallidas por el promedio o la media de la columna. La primera prueba que realizaremos será remplazar con el promedio, para simular que la lectura fue correcta y dio un valor dentro del promedio. Esto lo haremos en la sección de preprocesamiento.

### Revisar la columna Sex

In [ ]:
# Graficar Histograma de Sex
cirrhosis_Xtrain['Sex'].hist()

Revisando los datos, se nota una gran diferencia entre hombres y mujeres en la distribución, al tener una mayoria de mujeres significativa, se revisara con negocio si se mantiene o elimina la columna, y si queremos hacer solo predicciones con mujeres, de igual manera habrá que revisar la correlación de esta con el campo, para ver si realmente es significativa, lo cual se vera más adelante.

### Analysis univariado de variables numericas

#### Representación de histogrmas para visualizar las distirbuciones

In [ ]:
# Se evalúa si las variables númericas requieren alguna transformación (Excluytendo ID, que sabemos que no la necesitamos)
cat_con = cirrhosis_Xtrain.select_dtypes(include="number").columns.tolist()[1:]

fig, axes = plt.subplots(4,3, figsize=(50,40))
axes = axes.ravel()
for col, ax in zip(cirrhosis_Xtrain[cat_con], axes):
  sns.histplot(x=cirrhosis_Xtrain[col], ax=ax)
  ax.set(title=f'{col}', xlabel=None)

Observando los histogramas, podemos ver que hay algunas variables que cuentan con distribuciones no homogeneas, por lo que muy probablemente, exitan outlieres y debamos hacer alguna transformación para estas variables.Algunas de estas variables son Bilirubin, Cholesterol, Copper, Alk_Phos, SGOT, tryglicerides, Prothrombin. Las siguientes transformaciones se aplicaran en la sección de preprocesamiento:    

skew positiva o negativa. A los cuales pertenece "Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothtombin". A estas les aplicare las siguientes transformaciones SimpleInputer, Standard Scaler y Función raíz cuadrada. La función raíz cuadrada es para quitar el skew y el Standard Scaler es para normalizar la distribución y escalar los datos en caso de usar regresión logística.

Al resto de variables les aplicare solamente el imputer y el Standard Scaler para normalizar la distribución.

#### Diagramas de caja de variables numericas

In [ ]:
def boxplot(df):

  cat_con = cirrhosis_Xtrain.select_dtypes(include="number").columns.tolist()[1:]

  fig, axes = plt.subplots(4,3, figsize=(50,40))
  axes = axes.ravel()
  for col, ax in zip(cirrhosis_Xtrain[cat_con], axes):
    sns.boxplot(x=cirrhosis_Xtrain[col], ax=ax)
    ax.set(title=f'{col}', xlabel=None)

boxplot(df_cirrhosis)


Notamos que efectivamente las variables mencionadas tienen outliers, por lo que vamos a remover algunos de los outliers encontrados. Pero viendo la tendencias de las curvas, haremos las transformaciones mencionadas para evitar el efecto de los outliers significativamente.

### Analysis univariado de variables categoricas

#### Hacemos el countplot de las variables categoricas

In [ ]:
cat_con = cirrhosis_Xtrain.select_dtypes(include="object").columns.tolist()[1:]

fig, axes = plt.subplots(3,3, figsize=(50,40))
axes = axes.ravel()
for col, ax in zip(cirrhosis_Xtrain[cat_con], axes):
  sns.countplot(x=cirrhosis_Xtrain[col], ax=ax)
  ax.set(title=f'{col}', xlabel=None)

Observamos que las variables categoricas, están muy distintas en valores, por lo que valdría la pena verificar cuales si están realmente relacionadas con el valor a predecir. Lo cual veremos en el analisis multivariado de variables no numericas

### Analysis multivariado de variables numericas

#### Realizamos el mapa de correlación de Pearson para ver si existen variables que esten fuertemente relacionadas entre si, para descartarlas en el analisis.

In [ ]:
sns.set(rc={'figure.figsize':(13,13)})
cirrhosis_XtrainCorr = cirrhosis_Xtrain.copy().drop('ID',axis=1)
cirrhosis_XtrainCorr["Stage"] = cirrhosis_ytrain
sns.heatmap(cirrhosis_XtrainCorr.corr(numeric_only=True), annot=True,cmap="BuGn")

Observamos que no existe ninguna correlación fuerte negativa o positiva, por lo que decidimos mantener todas las variables en la primera iterración.

### Analysis multivariado de Variables no númericas

#### Realizamos Diagramas de caja para ver como se relacionan las variables con la variable de salida

In [ ]:
cat_con = cirrhosis_Xtrain.select_dtypes(include="object").columns.tolist()
df_para_graficar = Graf = pd.concat([cirrhosis_Xtrain[cat_con],cirrhosis_ytrain],axis=1)

fig, axes = plt.subplots(2,3, figsize=(50,40))
axes = axes.ravel()
for col, ax in zip(cirrhosis_Xtrain[cat_con], axes):
  sns.boxplot(Graf, x=col,y="Status",ax=ax)
  ax.set(title=f'{col}', xlabel=None)

Podemos observar que la columna sexo no tiene ninguna relevancia con la catgoria stage, por lo que se podria eliminar, ya que no aporta mucha información. Observamos que el resto de las columnas se pueden conservar

# Preprocesamiento

### Definir pasos del pipeline

#### Declarar imputer para remplazar los valores nulos con el promedio

In [86]:
cirrhosis_mean_imputer = SimpleImputer(strategy='mean')


#### Declarar Standard Scaler para Scalar los datos y estandarizar las distribuciones

In [87]:
cirrhosis_standard_scaler = StandardScaler()

#### Declarar Function transformer para ejecutar la función logaritmo y ajustar las distribuciones

In [88]:
def f(self, input_features):
    return input_features

cirrhosis_sqrt_transformer = FunctionTransformer(np.sqrt,feature_names_out=f)

#### Declarar OneHotEncoder para las variables categoricas

In [112]:
cirrhosis_one_hot_encoder = OneHotEncoder()

### Definir pipeline de datos

Definiremos el pipeline que aplicara las siguientes tareas:

Columnas Númericas con skew, que son las siguientes ("Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothtombin"):
    1.- Imputar valores con los promedios
    2.- Logaritmo para ajustar skew
    3.- Scaler para scalra y normalizar la distrubyucion

Columnas Numericas sin skew, que son las siguientes ("N_Days", "Age"):
    1.- Imputar valores con los promedios
    2.- Scaler para scalar y normalizar la distribución

Columnas no Numericas, que son las siguientes ("Drug", "Ascites","Hepatomegaly", "Spiders", Edema ):
    1.- OneHotEncoder

Quitaremos las siguientes columnas "ID", "Sex"
Stage es ordinal, por lo que se dejara como esta


In [113]:
cirrhosis_skew_numerico_columnas = ["Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets",]
cirrhosis_no_skew_numerico_columnas = ["N_Days","Age"]
cirrhosis_categoricas_columnas =  ["Drug","Sex","Ascites","Hepatomegaly","Spiders","Edema"]
columnas_a_eliminar = ['Sex']

cirrhosis_skew_numerico_pipeline = Pipeline(steps=[
    ('imputador',cirrhosis_mean_imputer),
    ('Logaritmo',cirrhosis_sqrt_transformer),
    ('Escalador',cirrhosis_standard_scaler)
    ])
cirrhosis_no_skew_numerico_pipeline = Pipeline(steps=[
    ('imputador',cirrhosis_mean_imputer),
    ('Escalador',cirrhosis_standard_scaler)
    ])
cirrhosis_categoricas_pipeline = Pipeline(steps=[
    ('codificador',cirrhosis_one_hot_encoder)
    ])

### Definir el transformador de columnas para aplicar de manera correcta las transformaciones

In [ ]:
cirrhosis_transformador_de_columnas = ColumnTransformer(transformers = [('numericas_con_skew', cirrhosis_skew_numerico_pipeline, cirrhosis_skew_numerico_columnas),('numericas_sin_skew', cirrhosis_no_skew_numerico_pipeline, cirrhosis_no_skew_numerico_columnas),('categoricas',cirrhosis_categoricas_pipeline,cirrhosis_categoricas_columnas),('Eliminar_Columnas','drop',columnas_a_eliminar)],remainder='passthrough',verbose_feature_names_out=False)
cirrhosis_transformador_de_columnas

### Entrenar el transformador con los datos de entrenamiento

In [122]:
cirrhosis_transformador_de_columnas_entrenado = cirrhosis_transformador_de_columnas.fit(cirrhosis_Xtrain)


### Preprocesar los datos de entrenamiento

In [ ]:
cirrhhosis_Xtrain_preprocesado = cirrhosis_transformador_de_columnas.transform(cirrhosis_Xtrain)
cirrhhosis_Xtrain_preprocesado_df = pd.DataFrame(
    cirrhhosis_Xtrain_preprocesado, columns=cirrhosis_transformador_de_columnas_entrenado.get_feature_names_out(),
    index=cirrhosis_ytrain.index)
cirrhhosis_Xtrain_preprocesado_df.head()

### Salvar los datos preprocesados de entrenamiento

In [147]:
cirrhosis_X_train_preprocesed_path = '../data/cirrhosis_X_train_preprocesed.csv'
cirrhhosis_Xtrain_preprocesado_df.to_csv(cirrhosis_X_train_preprocesed_path)

### Preprocesar los datos de validación

In [ ]:
cirrhhosis_Xval_preprocesado = cirrhosis_transformador_de_columnas.transform(cirrhosis_Xval)
cirrhhosis_Xval_preprocesado_df = pd.DataFrame(
    cirrhhosis_Xval_preprocesado, columns=cirrhosis_transformador_de_columnas_entrenado.get_feature_names_out(),
    index=cirrhosis_yval.index)
cirrhhosis_Xval_preprocesado_df.head()

### Salvar los datos de validación preprocesados

In [127]:
cirrhosis_X_val_preprocesed_path = '../data/cirrhosis_X_val_preprocesed.csv'
cirrhhosis_Xval_preprocesado_df.to_csv(cirrhosis_X_val_preprocesed_path)

### Preprocesar los datos de prueba

In [ ]:
cirrhhosis_Xtest_preprocesado = cirrhosis_transformador_de_columnas.transform(cirrhosis_Xtest)
cirrhhosis_Xtest_preprocesado_df = pd.DataFrame(
    cirrhhosis_Xtest_preprocesado, columns=cirrhosis_transformador_de_columnas_entrenado.get_feature_names_out(),
    index=cirrhosis_ytest.index)
cirrhhosis_Xtest_preprocesado_df.head()

### Salvar los datos preprocesados de prueba

In [129]:
cirrhosis_X_test_preprocesed_path = '../data/cirrhosis_X_test_preprocesed.csv'
cirrhhosis_Xtest_preprocesado_df.to_csv(cirrhosis_X_test_preprocesed_path)

# Entrenar Modelo

In [ ]:
model = LogisticRegression(max_iter=3000)
model.fit(cirrhhosis_Xtrain_preprocesado, cirrhosis_ytrain)

# Evaluar modelo

## Realizar predicciones

In [142]:
y_pred = model.predict(cirrhhosis_Xtrain_preprocesado)


## Matriz de confusion

In [ ]:
cm = confusion_matrix(cirrhosis_ytrain, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(cirrhosis_ytrain), yticklabels=np.unique(cirrhosis_ytrain))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## Precision

In [ ]:
scores = cross_val_score(model, cirrhhosis_Xtrain_preprocesado, cirrhosis_ytrain, cv=10)
print("Average accuracy with CV:", np.mean(scores))

# Salvar Modelo

In [146]:
path_del_modelo = "../models/cirrhosis_log_reg_model.sav"
pickle.dump(model, open(path_del_modelo, 'wb'))